In [182]:
import solana
from solana.publickey import PublicKey
from solana.rpc.api import Client
from solana.keypair import Keypair
from solana import system_program
from solana import transaction
from solana.system_program import SYS_PROGRAM_ID, CreateAccountParams, create_account
from solana.transaction import *
from spl.token.constants import TOKEN_PROGRAM_ID, ASSOCIATED_TOKEN_PROGRAM_ID
from spl.token.instructions import initialize_mint, InitializeMintParams
from spl.token._layouts import MINT_LAYOUT
from spl.token.client import Token
from spl.token import instructions
import base64
from borsh_construct import *
from solana.rpc.commitment import Commitment
import time


client = Client("https://api.devnet.solana.com")

client.is_connected()

True

In [183]:
program_id = PublicKey("DvTvXKZR9wveQzL3pBM7QJ9YZpConSXBXRTyDsEADNqX")
# program_id_account_meta = AccountMeta(program_id, False, False)
sys_program_meta = AccountMeta(SYS_PROGRAM_ID, False, False)
spl_program_meta  = AccountMeta(TOKEN_PROGRAM_ID, False, False)
# program_id_account_meta

In [184]:
# client.request_airdrop(PublicKey("DxkiJXK31VEd4R6F44mQRszoUQugpPYMwL9ybcrnt22r"), 2000000000)

In [185]:
def getAccountInfo(client: Client, account_pk: PublicKey):
    """Fetch account information from RPC, parse out the data and deserialize"""
    comm = Commitment("confirmed")
    res = client.get_account_info(account_pk, comm, encoding='base64')
    data = res['result']
    if isinstance(data, dict):
        return accountSchema.parse(base64.urlsafe_b64decode(data['value']['data'][0]))
    else:
        raise AttributeError(f'Unknown RPC result {data}')

In [186]:
seed = bytes([125,215,212,54,213,98,45,123]*4)
payer_keypair = Keypair.from_seed(seed)
# client.request_airdrop(payer_keypair.public_key, 2*10**9)

In [187]:
dsol_mint = PublicKey("4hnebtBZBkWF8NvPr4XUdKQFNj2tHim8nqephcMCsBos")

In [188]:
seed = bytes([25,215,212,154,213,198,45,123]*4)
vault_keypair = Keypair.from_seed(seed)
# vault_keypair = Keypair.generate()
# client.request_airdrop(vault_keypair.public_key, 2*10**9)
# time.sleep(5)

vault_dsol_token_account = instructions.get_associated_token_address(vault_keypair.public_key, dsol_mint)

In [189]:
mint_account = Keypair.generate()

payer_dsol_token_account = instructions.get_associated_token_address(payer_keypair.public_key, dsol_mint)
payer_account_meta = AccountMeta(payer_keypair.public_key, True, True)
vault_account_meta = AccountMeta(vault_keypair.public_key, True, True)

rent_account_meta = AccountMeta(solana.sysvar.SYSVAR_RENT_PUBKEY, False, False)
spl_program_meta  = AccountMeta(TOKEN_PROGRAM_ID, False, False)


In [190]:
mint_account_meta = AccountMeta(mint_account.public_key, True, True)
associated_token_address = instructions.get_associated_token_address(payer_keypair.public_key, mint_account.public_key)
associated_account_meta = AccountMeta(associated_token_address, False, True)

(mint_authority, nonce) = PublicKey.find_program_address([b"avatar_mint_authority_pda"], program_id)
(avatar_data_pda, avatar_pda_nonce) = PublicKey.find_program_address([b'avatar_data_pda', bytes(mint_account_meta.pubkey)], program_id)
(sales_pda, sales_nonce) = PublicKey.find_program_address([b'sales_pda'], program_id)
metadata_program_pubkey = PublicKey("metaqbxxUerdq28cj1RbAWkYQm3ybzjb6a8bt518x1s")
(metadata_pda, metadata_nonce) = PublicKey.find_program_address([b'metadata', bytes(metadata_program_pubkey), bytes(mint_account_meta.pubkey)], metadata_program_pubkey)


avatar_data_pda_meta = AccountMeta(avatar_data_pda, False, True)
sysvar_clock_meta = AccountMeta(solana.sysvar.SYSVAR_CLOCK_PUBKEY, False, False)
sales_pda_meta = AccountMeta(sales_pda, False, True)
mint_authority_meta = AccountMeta(mint_authority, False, False)
associated_program_meta = AccountMeta(ASSOCIATED_TOKEN_PROGRAM_ID, False, False)


dsol_mint_meta = AccountMeta(dsol_mint, False, False)
vault_dsol_token_account_meta = AccountMeta(vault_dsol_token_account, False, True)
payer_dsol_token_account_meta = AccountMeta(payer_dsol_token_account, False, True)

print(dsol_mint)
print(vault_keypair.public_key)
print(vault_dsol_token_account)
print(payer_keypair.public_key)
print(payer_dsol_token_account)
print(sys_program_meta)
print(spl_program_meta)
print(rent_account_meta)
print(associated_token_address)

4hnebtBZBkWF8NvPr4XUdKQFNj2tHim8nqephcMCsBos
3HqXocfan3tVv3N4tTAFhyVePkP856jxngQ6pQxdupXP
9ZJkPCjEZehchPHs1k327BCb9JQVEg7drn7nmkK6Wkqk
2ASw3tjK5bSxQxFEMsM6J3DnBozNh7drVErSwc7AtzJv
4JTdYhzx6rorDjPMGdXDQCLuF3nM8dpiodnUgqqqhHEc
AccountMeta(pubkey=11111111111111111111111111111111, is_signer=False, is_writable=False)
AccountMeta(pubkey=TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA, is_signer=False, is_writable=False)
AccountMeta(pubkey=SysvarRent111111111111111111111111111111111, is_signer=False, is_writable=False)
EqnuhKqFELYnhETBsivBxgSVVHPgkxLv2C6GYzxnMf1L


In [191]:
def init_vault():
    accounts = [
        payer_account_meta,
        vault_account_meta,
        vault_dsol_token_account_meta,
        dsol_mint_meta,
        sys_program_meta,
        spl_program_meta,
        rent_account_meta,

        associated_program_meta,
        sys_program_meta,
        sys_program_meta,
    ]

    transaction = Transaction()
    transaction.add(
        TransactionInstruction(
            accounts,
            program_id,
            bytes([9])
        )
    )
    
    return f"https://explorer.solana.com/tx/{client.send_transaction(transaction, payer_keypair, vault_keypair)['result']}?cluster=devnet"

# init_vault()

# time.sleep(5)

In [192]:
program_id

DvTvXKZR9wveQzL3pBM7QJ9YZpConSXBXRTyDsEADNqX

In [193]:
def mint_avatar():
    accounts = [
        # program_id_account_meta,
        payer_account_meta,
        payer_dsol_token_account_meta,
        vault_account_meta, 
        vault_dsol_token_account_meta,
        mint_account_meta, 
        rent_account_meta, 
        associated_account_meta,
        spl_program_meta,
        AccountMeta(SYS_PROGRAM_ID, False, False),
        mint_authority_meta,
        avatar_data_pda_meta,
        sales_pda_meta,
        AccountMeta(metadata_pda, False, True),
        sysvar_clock_meta,

        spl_program_meta,
        AccountMeta(ASSOCIATED_TOKEN_PROGRAM_ID, False, False),
        spl_program_meta,
        spl_program_meta,
        AccountMeta(metadata_program_pubkey, False, False),
    ]   

    transaction = Transaction()
    transaction.add(TransactionInstruction(
        accounts,
        program_id,
        bytes([0])
    ))
    
    return f"https://explorer.solana.com/tx/{client.send_transaction(transaction, payer_keypair, mint_account, vault_keypair)['result']}?cluster=devnet"

mint_avatar()

'https://explorer.solana.com/tx/3pxpcoMJX83vpmPnb9iAX78dNL2bHJ6XbLsAow9QyepjzSZe86u2p3Wk8SvgtovUo2HAv5Kc1bvPrR2kW7oir2V?cluster=devnet'

In [194]:

accountSchema = CStruct(
    "vault_total" / F32,
    "counter" / U32,
    
    "rent_min_listed"/ U32,
    "rent_max_listed"/ U32,
    "rent_max_ever"/ U32,
)

(sales_pda, sales_nonce) = PublicKey.find_program_address([b'sales_pda'], program_id)
print(f"PDA: {sales_pda, sales_nonce}")

getAccountInfo(client,sales_pda)

PDA: (AxjHKB6QWUWFSefbAwAx5UZggfZZkzpAu7Gs94U8sGAU, 255)


Container(vault_total=17.92838478088379, counter=11, rent_min_listed=0, rent_max_listed=0, rent_max_ever=0)

In [195]:
accountSchema = CStruct(
    "date_created" / U32,
    "unlockable_date" / U32,
    "numeration" /U32,
    "rarity" / U8,
    "level" / U8,
    "xp" / U32,
    "rented_state" / Bool,
    "use_authority" / U8[32],
    "rent_bump" / U32,
)

print(f"PDA: {avatar_data_pda, avatar_pda_nonce}")

# PublicKey(getAccountInfo(client,avatar_data_pda).use_authority)
getAccountInfo(client,avatar_data_pda)

PDA: (EqoGmy4c6Npx8FYwA3eDs4rJzvhLRekbfGidsPpwDUv3, 255)


Container(date_created=1645204021, unlockable_date=1645204021, numeration=10, rarity=1, level=0, xp=0, rented_state=False, use_authority=ListContainer([17, 71, 60, 73, 89, 58, 122, 18, 156, 76, 53, 19, 249, 230, 254, 224, 26, 170, 93, 110, 57, 38, 59, 79, 2, 89, 12, 49, 56, 233, 39, 27]), rent_bump=0)

In [196]:
# time.sleep(120)

# new_accounts = [
#     mint_account_meta,
#     associated_account_meta,
#     spl_program_meta,
#     mint_authority_meta,
#     avatar_data_pda_meta,
#     sysvar_clock_meta,

#     spl_program_meta
# ]

# transaction = Transaction()
# transaction.add(TransactionInstruction(
#     new_accounts,
#     program_id,
#     bytes([1])
# ))
# f"https://explorer.solana.com/tx/{client.send_transaction(transaction, payer_keypair, mint_account)['result']}?cluster=devnet"